sequence-to-sequence학습이란?
>> 이 모델은 기계 번역 혹은 자유로운 질의응답에 사용
>> 자연어 질문을 주어 자연어 응답을 생성
>> 일반적으로, 텍스트를 생성해야 할 경우라면 언제든지 적용가능

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras import optimizers, losses, metrics

import numpy as np


#  데이터 생성

In [2]:
# 캐릭터 글자 목록
# S -> Start, E -> End, P -> Padding
char_list = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz사랑얼굴주택희망나무바위']

# 캐릭터 사전 생성
char_to_idx = {c: i for i, c in enumerate(char_list)}
dic_len = len(char_to_idx)

print(char_to_idx)
print(dic_len)


{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '사': 29, '랑': 30, '얼': 31, '굴': 32, '주': 33, '택': 34, '희': 35, '망': 36, '나': 37, '무': 38, '바': 39, '위': 40}
41


In [3]:
# 단어 목록
# 앞의 영어를 뒤의 한글로 번역
word_list = [['love', '사랑'], ['face', '얼굴'],
            ['home', '주택'], ['hope', '희망'],
            ['tree', '나무'], ['rock', '바위']]

In [4]:
encoder_input = []
decoder_input = []
decoder_target = []

In [5]:
for words in word_list:
    enc_input = [char_to_idx[c] for c in words[0]]
    encoder_input.append(np.eye(dic_len)[enc_input])
np.array(encoder_input).shape # 단어 6개, time_step=4, 단어사전길이:41

(6, 4, 41)

In [6]:
# 배치 데이터 생성
def make_batch(word_list):
    
    encoder_input = []
    decoder_input = []
    decoder_target = []

    for words in word_list:
        # 인코더 입력 단어를 인덱스로 변환
        # Ex) l,o,v,e
        enc_input = [char_to_idx[c] for c in words[0]]

        # 디코더 입력 단어를 인덱스로 변환
        # 제일 앞에 시작 태그 삽입
        # Ex) S,사,랑
        dec_input = [char_to_idx[c] for c in ('S' + words[1])]
        
        # 디코더 목표 캐릭터를 인덱스로 변환
        # 제일 끝에 종료 태그 삽입
        # Ex) 사,랑,E
        dec_target = [char_to_idx[c] for c in (words[1] + 'E')]

        # 원핫인코딩으로 변환
        encoder_input.append(np.eye(dic_len)[enc_input])
        decoder_input.append(np.eye(dic_len)[dec_input])
        decoder_target.append(np.eye(dic_len)[dec_target])

    return np.array(encoder_input), np.array(decoder_input), np.array(decoder_target)

In [7]:
# 배치 생성
x_encoder, x_decoder, y_decoder = make_batch(word_list)

In [8]:
x_encoder[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
x_decoder[0] # s, 사, 랑


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
y_decoder[0] # 사, 랑, E


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# 모델 생성

In [12]:
#--------------------------------------------
# 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = Input(shape=(None,dic_len)) # time_step None으로 받기, 문자 길이는 원래 정해져 있지 않음

# return_state가 True면 상태값 리턴
# return_state는 encoder의 출력과 내부 RNN상태인 리스트를 반환하도록 RNN을 구성하는 인수
# 이는 encoder의 상태를 복구하는 데 사용
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = LSTM(64,
                                        dropout=0.1,
                                        recurrent_dropout=0.5,
                                        return_state=True)(encoder_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
# time_step = None
decoder_inputs = Input(shape=(None,dic_len))

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = LSTM(64,
                    dropout=0.1,
                    recurrent_dropout=0.5,
                    return_state=True,
                    return_sequences=True)

# initial_state를 인코더의 상태로 초기화
# initial_state는 RNN의 초기 상태를 지정하는 인수. 초기 상태로 encoder를 decoder로 전달하는 데 사용
# input은 decoder_inputs, encoder_states
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = Dense(dic_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

In [13]:
# 훈련 시작
model.fit([x_encoder, x_decoder],
           y_decoder,
           epochs=200,
           batch_size=1)

Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 3.6865 - acc: 0.1111
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 3.6201 - acc: 0.3333
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 3.5620 - acc: 0.3333
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 3.4649 - acc: 0.3333
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 3.3476 - acc: 0.3333
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 3.1845 - acc: 0.3333
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 2.9562 - acc: 0.3333
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 2.5727 - acc: 0.3333
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 2.4364 - acc: 0.3333
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 2.3344 - acc: 0.3333
Epoch 11/200
6/6 [==============================] - 0s 2ms/step - loss: 2.3004 

In [14]:
# 번역 수행
def translate(word):
    
    # 영어/한글 배열 생성
    # 한글은 정답을 모르기 때문에 패딩으로 채움
    words = [word, 'P' * len(word)]

    # 배치 데이터 생성
    x_encoder, x_decoder, y_decoder = make_batch([words])
    
    # 예측 수행
    # 원핫인코딩으로 결과 나옴
    results = model.predict([x_encoder, x_decoder])

    # 2축을 기준으로 최대값의 인덱스 구함
    results = np.argmax(results, 2) 

    # 인덱스를 캐릭터로 변환
    decoded = [char_list[i] for i in results[0]]

    # 종료 태그인 'E' 이후의 글자들을 제거하고 문자열 생성
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [15]:
words = ['love','P'*len('word')]
x_encoder, x_decoder, y_decoder = make_batch([words])
y_decoder

array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [16]:
results = model.predict([x_encoder,x_decoder])
results = np.argmax(results,2)
results # 사, 랑 , E

array([[29, 30,  1,  1,  1]], dtype=int64)

x_encoder = 'love'<br>
x_decoder = 'SPPPP'<br>
y_decoder = 'PPPPE'

In [ ]:
print('love ->', translate('love'))
print('lovi ->', translate('lovi'))
print('litr ->', translate('litr'))
print('hope ->', translate('hope'))
print('hopu ->', translate('hopu'))
print('hufe ->', translate('hufe'))
